In [20]:
import os
import json

MODEL_PATH = '/beegfs/ci411/pcen/models/'

In [23]:
models = os.listdir(MODEL_PATH)
results = {}
for model in models:
    try:
        with open(os.path.join(MODEL_PATH, model, 'results.json'), 'r') as fd:
            result = json.load(fd)
        results[model] = result
    except FileNotFoundError:
        print("No results.json file for model '{}'".format(model))
        results[model] = None


No results.json file for model 'model_test'


In [24]:
results

{'full_test': {'event_based_metrics': {'class_wise': {'air_conditioner': {'accuracy': {},
     'count': {'Nref': 838.0, 'Nsys': 3740.0},
     'error_rate': {'deletion_rate': 0.9415274463007159,
      'error_rate': 5.3460620525059666,
      'insertion_rate': 4.40453460620525},
     'f_measure': {'f_measure': 0.021406727828746176,
      'precision': 0.013101604278074866,
      'recall': 0.05847255369928401}},
    'car_horn': {'accuracy': {},
     'count': {'Nref': 934.0, 'Nsys': 1848.0},
     'error_rate': {'deletion_rate': 0.791220556745182,
      'error_rate': 2.5610278372591004,
      'insertion_rate': 1.7698072805139187},
     'f_measure': {'f_measure': 0.14018691588785046,
      'precision': 0.10551948051948051,
      'recall': 0.20877944325481798}},
    'children_playing': {'accuracy': {},
     'count': {'Nref': 861.0, 'Nsys': 4117.0},
     'error_rate': {'deletion_rate': 0.9407665505226481,
      'error_rate': 5.663182346109175,
      'insertion_rate': 4.722415795586527},
     'f_